In [45]:
import featuretools as ft
import pandas as pd
from datetime import datetime

In [46]:
transactions_df = pd.DataFrame({"transaction_id": ["001", "002", "003", "004", "005", "006", "007", "008"],
                                "user_id": [1, 2, 1, 3, 2, 5, 5, 2],
                                "amount": [100.40, 20.63, 33.32, 13.12, 67.22, 1.00, 3.00, 20.99],
                                "transaction_time": pd.date_range(start="2014-01-01 08:00:50", periods=8, freq="12h"),
                                "name": ["Diana Marshall", "Stewart Alsop", "Diana Marshall", "Emma Ball", "Stewart Alsop", "Penelope Walsh", "Penelope Walsh", "Stewart Alsop"],
                                "user_credit": [810, 730, 810, 690, 730, 520, 520, 730]})

In [47]:
transactions_df

,transaction_id,user_id,amount,transaction_time,name,user_credit
0,001,1,100.40,2014-01-01 08:00:50,Diana Marshall,810
1,002,2,20.63,2014-01-01 20:00:50,Stewart Alsop,730
2,003,1,33.32,2014-01-02 08:00:50,Diana Marshall,810
3,004,3,13.12,2014-01-02 20:00:50,Emma Ball,690
4,005,2,67.22,2014-01-03 08:00:50,Stewart Alsop,730
5,006,5,1.00,2014-01-03 20:00:50,Penelope Walsh,520
6,007,5,3.00,2014-01-04 08:00:50,Penelope Walsh,520
7,008,2,20.99,2014-01-04 20:00:50,Stewart Alsop,730


In [61]:
transactions_df.dtypes

transaction_id              object
user_id                      int64
amount                     float64
transaction_time    datetime64[ns]
name                        object
user_credit                  int64
dtype: object

In [53]:
es = ft.EntitySet()

In [54]:
es = es.entity_from_dataframe(entity_id="transactions",
                               dataframe=transactions_df,
                               index="transaction_id",
                               time_index="transaction_time")

In [55]:
es = es.normalize_entity(base_entity_id="transactions",
                          new_entity_id="users",
                          index="user_id",
                          make_time_index=False,
                          additional_variables=["name", "user_credit"])

In [56]:
ct = pd.DataFrame()
ct['user_id'] = [1, 2, 3, 5]

ct['time'] = pd.to_datetime(['2014-01-02 08:00:50',
                            '2014-01-02 08:00:50',
                            '2014-01-02 20:00:50',
                            '2014-01-03 12:00:50'])
ct['label'] = [False, False, False, True]

# This demos single entity normalize functionality


In [59]:
# fm, features = ft.dfs(entityset=es,
#                      target_entity='transactions',
#                       trans_primitives=["year", "month", "weekday", "percentile"],
#                       max_depth=1)

fm, features = ft.dfs(entityset=es,
                     target_entity='users',
                     trans_primitives=["year", "month", "weekday", "percentile"],
                     cutoff_time=ct,
                     # max_depth=1,
                     cutoff_time_in_index=True)

In [60]:
fm

,,name,user_credit,SUM(transactions.amount),STD(transactions.amount),MAX(transactions.amount),SKEW(transactions.amount),MIN(transactions.amount),MEAN(transactions.amount),COUNT(transactions),PERCENTILE(user_credit),...,MODE(transactions.MONTH(transaction_time)),MODE(transactions.WEEKDAY(transaction_time)),PERCENTILE(SUM(transactions.amount)),PERCENTILE(STD(transactions.amount)),PERCENTILE(MAX(transactions.amount)),PERCENTILE(SKEW(transactions.amount)),PERCENTILE(MIN(transactions.amount)),PERCENTILE(MEAN(transactions.amount)),PERCENTILE(COUNT(transactions)),label
user_id,time,,,,,,,,,,,,,,,,,,,,,
1,2014-01-02 08:00:50,Diana Marshall,810,133.72,47.432723,100.40,NaN,33.32,66.86,2.0,1.00,...,1.0,2.0,1.00,1.0,1.000000,NaN,1.000000,1.000000,1.000,False
2,2014-01-02 08:00:50,Stewart Alsop,730,20.63,NaN,20.63,NaN,20.63,20.63,1.0,0.75,...,1.0,2.0,0.75,NaN,0.500000,NaN,0.500000,0.500000,0.750,False
3,2014-01-02 20:00:50,Emma Ball,690,13.12,NaN,13.12,NaN,13.12,13.12,1.0,0.50,...,1.0,3.0,0.50,NaN,0.333333,NaN,0.333333,0.333333,0.625,False
5,2014-01-03 12:00:50,Penelope Walsh,520,0.00,NaN,NaN,NaN,NaN,NaN,0.0,0.25,...,NaN,NaN,0.25,NaN,NaN,NaN,NaN,NaN,0.250,True
